In [112]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import KNNImputer

In [113]:
# df = pd.read_excel(r"C:\Users\Admin\data stb\mock survey data 3.1.xlsx") #desktop
df = pd.read_excel(r"C:\Users\alvin\Tourism assignment\mock survey data 3.1.xlsx")

In [114]:
df

,case,year,month,Country of Residence,City of Residence,Purpose of Visit,Main Purpose of Visit,Weights_QTR,Air_Terminal,Sea_Terminal,...,Travel companion - Alone,Travel companion - Spouse,Travel companion - Your Child/Children,Travel companion - Parents/Parents-in-law,Travel companion - Grandparents/Grandparents-in-law,Travel companion - Siblings,Travel companion - Other relatives,Travel companion - Friends,Travel companion - Business associates/Colleagues,Travel companion - Others
0,1.0,2018,1,India,Delhi,Leisure,Holiday/ Rest & Relax,660.046512,2.0,NaN,...,0,1,0,0,0,0,0,0,0,0
1,2.0,2018,1,India,Delhi,Leisure,Visiting friends/ relatives (who are not inter...,433.217949,2.0,NaN,...,1,0,0,0,0,0,0,0,0,0
2,3.0,2018,1,United Arab Emirates,Dubai,Leisure,General business purpose,335.125000,2.0,NaN,...,1,0,0,0,0,0,0,0,0,0
3,4.0,2018,1,India,Others,Leisure,Holiday/ Rest & Relax,490.977273,2.0,NaN,...,0,1,1,0,0,0,0,0,0,0
4,5.0,2018,1,Canada,Toronto,Business + Accompanying Pax,Visiting an international student (relative or...,379.800000,2.0,NaN,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22969,7789.0,2018,12,Australia,Brisbane,Leisure,Holiday/ Rest & Relax,310.354839,2.0,NaN,...,0,1,0,0,0,0,0,0,0,0
22970,7790.0,2018,12,Hong Kong,Hong Kong,Leisure,Holiday/ Rest & Relax,109.393939,2.0,NaN,...,0,0,0,0,0,0,0,1,0,0
22971,7791.0,2018,12,Hong Kong,Hong Kong,Leisure,Holiday/ Rest & Relax,326.906250,2.0,NaN,...,0,1,0,0,0,0,0,0,0,0
22972,7792.0,2018,12,Hong Kong,Hong Kong,Business + Accompanying Pax,Gather information/ facts on the education ser...,820.615385,2.0,NaN,...,1,0,0,0,0,0,0,0,0,0


In [115]:
# Overview
print("Data Shape:", df.shape)
num_columns = len(df.columns)
print("Total number of columns:", num_columns)

print("\n")
print("Data Columns:", df.columns)



print("\n")
print("\nColumn Information:")
df.info()

Data Shape: (22974, 51)
Total number of columns: 51


Data Columns: Index(['case', 'year', 'month', 'Country of Residence', 'City of Residence',
       'Purpose of Visit', 'Main Purpose of Visit', 'Weights_QTR',
       'Air_Terminal', 'Sea_Terminal', 'Land_Terminal',
       'First Visit To Singapore', 'Length of Stay', 'Travel Type', 'Gender',
       'Marital Status', 'Occupation', 'Occupation.1', 'Other Designation',
       'Designation (free text)', 'shopping_fash', 'shopping_jewllery',
       'shopping_watches', 'shopping_wellness', 'shopping_food',
       'shopping_gifts', 'shopping_consumertech', 'shopping_antiques',
       'shopping_others', 'shopping_any', 'totacc', 'totfnb', 'tottran',
       'totbiz', 'totedu', 'totmedi', 'tototh', 'totshopping_rep', 'tot.exp',
       'c4a_1', 'MainHotel', 'Travel companion - Alone',
       'Travel companion - Spouse', 'Travel companion - Your Child/Children',
       'Travel companion - Parents/Parents-in-law',
       'Travel companion - Grand

In [116]:
# Define expected number of rows
expected_rows = 22974

# Initialize lists to store column names
full_columns = []
incomplete_columns = []

# Loop through each column
for col in df.columns:
    non_null_count = df[col].notnull().sum()
    if non_null_count == expected_rows:
        full_columns.append(col)
    else:
        incomplete_columns.append((col, non_null_count))

# Print columns with all values
print("**Columns with FULL data (22,974 rows):**")
for col in full_columns:
    print(f"- {col}")

print("\n")

print("**Columns with MISSING data:**")
for col, count in incomplete_columns:
    print(f"- {col}: {count} non-null values")


**Columns with FULL data (22,974 rows):**
- year
- month
- Country of Residence
- Main Purpose of Visit
- Weights_QTR
- First Visit To Singapore
- Length of Stay
- Travel Type
- Gender
- Marital Status
- Occupation
- shopping_jewllery
- shopping_watches
- shopping_wellness
- shopping_consumertech
- shopping_antiques
- totacc
- totfnb
- tottran
- totbiz
- totedu
- totmedi
- tototh
- totshopping_rep
- tot.exp
- c4a_1
- Travel companion - Alone
- Travel companion - Spouse
- Travel companion - Your Child/Children
- Travel companion - Parents/Parents-in-law
- Travel companion - Grandparents/Grandparents-in-law
- Travel companion - Siblings
- Travel companion - Other relatives
- Travel companion - Friends
- Travel companion - Business associates/Colleagues
- Travel companion - Others


**Columns with MISSING data:**
- case: 22972 non-null values
- City of Residence: 22962 non-null values
- Purpose of Visit: 20641 non-null values
- Air_Terminal: 17977 non-null values
- Sea_Terminal: 2381 non-

In [117]:
print("\nMissing Values per Column:")
print(df.isnull().sum())


Missing Values per Column:
case                                                       2
year                                                       0
month                                                      0
Country of Residence                                       0
City of Residence                                         12
Purpose of Visit                                        2333
Main Purpose of Visit                                      0
Weights_QTR                                                0
Air_Terminal                                            4997
Sea_Terminal                                           20593
Land_Terminal                                          20359
First Visit To Singapore                                   0
Length of Stay                                             0
Travel Type                                                0
Gender                                                     0
Marital Status                                           

## Data dictionary

- case - **Unique respondent ID**
- year - **Date of survey response (year)**
- month -	**Date of survey response (year and month)**
- Country of Residence- **Visitor’s country of origin**
- City of Residence	- **Visitor’s home city (if provided)**
- Main Purpose of Visit -	**Primary reason for their trip (e.g. Leisure, Business, MICE)**
- Weights_QTR - **Survey weight to project sample to population**
- Air_Terminal - **Mode of travel - Airplane**
- Sea_Terminal - **Mode of travel - Sea**
- Land_Terminal - **Mode of travel - Land**
- First Visit To Singapore - **Whether this is their first trip to Singapore (Yes/No)**
- Length of Stay - **Number of nights stayed**
- Travel Type - **Non package tour vs. packaged tour**
- Gender
- Marital Status
- Occupation 
- shopping_fash - **Spend on Fashion and Accessories**
- shopping_jewllery - **Spend on Genuine Jewellery**
- shopping_watches - **Spend on Watches and Clocks**
- shopping_wellness - **Spend on Healthcare and Wellness Products + Cosmetics, Perfumes and Toiletries**
- shopping_food - **Spend on Confectionery and Food Items**
- shopping_gifts - **Spend on Souvenirs and Gift**
- shopping_consumertech - **Spend on Consumer Tech**
- shopping_antiques - **Spend on Art and Collectors Items + Furniture, Carpets and Other Household Accessories**
- shopping_others - **shopping others**
- shopping_any - **shopping any**
- totacc - **Total spend on accommodation**
- totfnb - **Total spend on food & beverage**                                        
- tottran - **Total spend on transport**                                                           
- totbiz - **Total spend on Total Business Spend**                                                     
- totedu - **Total spend on Total Education Spend**                                                  
- totmedi - **Total spend on Total Medical Spend**                    
- tototh - **Total spend on Total Other Spend**                 
- totshopping_rep - **Total spend on Total Reported Shopping Spend (uses the shopping variables e.g. shopping_fash etc to calucate this)**
- tot.exp - **Total spend on 	Total Expenditure across all categories**
- c4a_1 **whether column of hotel names has been filled**
- MainHotel	- **Categorical code for hotel class or brand**
- Travel companion – **Binary for each companion type (alone, spouse, Your child/children, Parents/Parents-in-law, Grandparents/Grandparents-in-law,
  Siblings, Other relatives, Friends, Business associates/Colleagues, Others.)**



## Problem statement

Candidate Name:
Please note the following:

1)	Both questions use the provided datasets
  
2)	All source code must be provided with comments along with instructions on how to execute the code.
  
3)	R is preferred, but you are free to consider other tools. Regardless of the tool you use, source code / workings must be provided and clearly documented

  
4)	Refer to mock survey data 3.1.xls for Question 1 and 2.

  
5)	You are free to augment this dataset with publicly available data to sharpen your insights / recommendations.  However, if you choose to do so, you must document the reasons and how you are using the data to augment your insights / recommendations.

  
6)	You can refer to the sheet “dictionary” for a description of the fields to aid in your analysis.

Background
The marketing team is looking to refresh their marketing strategy and reached out to the Data Science team for help.  A survey was commissioned to collect visitor data over a period of 12 months to understand **their spending behaviours and preferences**.  You are now in-charge of analysing this data and presenting visitor behaviour insights to help the inform the marketing team on their strategy to increase visitorship and spend in Singapore.

Question 1

**With reference to the mock survey dataset (mock survey data 3.xlsx),** 

a.	Describe the analytical approach you will take and data fields you would look into when it comes to doing exploratory data analysis.

b.	Highlight the data idiosyncrasies / issues you found in this dataset and how would you address the issues identified.

c.	What are the considerations that you will take when analysing survey data.

Use this data set to illustrate and explain your approach and considerations.  Please include your supporting scripts/output from R or an alternative tool of your choice. 

Question 2

Using the mock survey data (mock survey data 3.1.xlsx), answer the following questions.

1)	**You are given a set of survey data which captures spend amounts among other data points.  The Marketing team is curious to understand the different segments of visitors who are coming to Singapore. You are given the task to analyse the survey data so that the marketing team can formulate data driven strategies to draw in more visitors to Singapore.**

Using the set of data provided, answer ALL of the following questions:

a.	How many different segment of visitors can be identified from the survey data

b.	Please explain the choice of metric you used

c.	What are the key assumptions you have taken to identify the different segments?

d.	What were the other approaches you considered? Please explain the reason for the technique / approach used as well as the pros and cons.

e.	Why is your analytical approach performing well / not well?

f.	Was any feature engineering required? If yes, what were they. If no, why?

 2)	**Based on the analysis done in (1), answer the following business questions:**
  
a.	Describe and profile the segment tourists that you have identified

b.	What can we learn about our visitors from the survey data as well as the visitor segments that will help the Marketing team better market Singapore as a tourist destination them?

c.	Is there a strong or weak relationship between travel companions and choice of hotel?  Please provide an explanation on how you arrived at your conclusion.

3)	**With your findings from (Questions 1, 2.1 and 2.2), prepare a concise PowerPoint presentation to pitch your strategy to the Marketing Team on how will your insights help STB attract visitors to spend in Singapore. Hints:**
•	You are encouraged to explore the data beyond the questions asked in (a) – (c)

•	You should use your analysis results from question 1 to support your recommendations

•	Use appropriate charts / visuals to communicate your story

•	Be careful with the chart-ink ratio


## 1. Describe the analytical approach you will take and data fields you would look into when it comes to doing exploratory data analysis.
Understand the the shape of the dataset, and what missing values are found in the dataset. Understand the data structure of each of the variable.
Variables are recoded (changing to datetime, categorical variables) after observations and running different test. Lastly, formulate possible problem statement to answer. Finding various variables to transform - e.g. age band, length of stay etc.

## 2. Highlight the data idiosyncrasies / issues you found in this dataset and how would you address the issues identified.
The data missing values for cases that doesnt have air, sea, land terminal - suggesting that there is possiblity of lack of proper documentation, remove these 3 participants. 

## **Columns with FULL data (22,974 rows):**
- year
- month
- Country of Residence
- Main Purpose of Visit
- Weights_QTR
- First Visit To Singapore
- Length of Stay
- Travel Type
- Gender
- Marital Status
- Occupation
- shopping_jewllery
- shopping_watches
- shopping_wellness
- shopping_consumertech
- shopping_antiques
- totacc
- totfnb
- tottran
- totbiz
- totedu
- totmedi
- tototh
- totshopping_rep
- tot.exp
- c4a_1
- Travel companion - Alone
- Travel companion - Spouse
- Travel companion - Your Child/Children
- Travel companion - Parents/Parents-in-law
- Travel companion - Grandparents/Grandparents-in-law
- Travel companion - Siblings
- Travel companion - Other relatives
- Travel companion - Friends
- Travel companion - Business associates/Colleagues
- Travel companion - Others


**Columns with MISSING data:**
- case: 22972 non-null values (remove column)
- City of Residence: 22962 non-null values (12 cases, will change them to others)
- Purpose of Visit: 20641 non-null values (remove this column, given that the dataset has main purpose of visit)
- Air_Terminal: 17977 non-null values (change to new variable 1, air travel)
- Sea_Terminal: 2381 non-null values (change to new variable 2, sea travel)
- Land_Terminal: 2615 non-null values (change to new variable 3, land travel)
- Occupation.1: 2790 non-null values (remove column, occupation is found in another column)
- Other Designation: 2789 non-null values (remove column, occupation is found in another column)
- Designation (free text): 192 non-null values (remove column, occupation is found in another column)
- shopping_fash: 22889 non-null values (k nearest neighbor imupte reason being that they may be correlated)
- shopping_food: 22968 non-null values (k nearest neighbor imupte reason being that they may be correlated)
- shopping_gifts: 22972 non-null values (k nearest neighbor imupte reason being that they may be correlated)
- shopping_others: 22971 non-null values (k nearest neighbor imupte reason being that they may be correlated)
- shopping_any: 22878 non-null values (k nearest neighbor imupte reason being that they may be correlated)
- MainHotel: 16284 non-null values (for correlation hotel will need to just include only people who have filled up hotel, sample enough to test various assumptions)

In [123]:
df[['shopping_fash', 'shopping_food', 'shopping_gifts', 'shopping_others', 'shopping_any']].nunique()

shopping_fash      420
shopping_food      274
shopping_gifts     249
shopping_others    257
shopping_any       914
dtype: int64

In [124]:
# Preprocessing data - part 1 - impute 'shopping_fash', 'shopping_food', 'shopping_others', 'shopping_any' 

df_trans = df.copy(deep=True)
impute_cols = ['shopping_fash', 'shopping_food', 'shopping_gifts', 'shopping_others', 'shopping_any']

knn_imputer = KNNImputer(n_neighbors=5, weights="uniform")
df_trans[impute_cols] = knn_imputer.fit_transform(df_trans[impute_cols])

In [125]:
df_trans[['shopping_fash', 'shopping_food', 'shopping_gifts', 'shopping_others', 'shopping_any']].isnull().sum()

shopping_fash      0
shopping_food      0
shopping_gifts     0
shopping_others    0
shopping_any       0
dtype: int64

In [126]:
# Filter rows where missing city of residence
no_city_resid_df = df_trans[df_trans['City of Residence'].isnull()] 

# Show the result
print(f"Number of rows with NO City of Residence: {len(no_city_resid_df)}")
display(no_city_resid_df)


Number of rows with NO City of Residence: 12


,case,year,month,Country of Residence,City of Residence,Purpose of Visit,Main Purpose of Visit,Weights_QTR,Air_Terminal,Sea_Terminal,...,Travel companion - Alone,Travel companion - Spouse,Travel companion - Your Child/Children,Travel companion - Parents/Parents-in-law,Travel companion - Grandparents/Grandparents-in-law,Travel companion - Siblings,Travel companion - Other relatives,Travel companion - Friends,Travel companion - Business associates/Colleagues,Travel companion - Others
4644,10314.0,2018,3,Others,NaN,Leisure,Visiting friends/ relatives (who are not inter...,510.800000,3.0,NaN,...,0,1,0,0,0,0,0,0,0,0
7543,13213.0,2018,4,Others,NaN,Leisure,Holiday/ Rest & Relax,685.666667,1.0,NaN,...,0,1,0,0,0,0,0,0,0,0
7955,13625.0,2018,5,Others,NaN,Leisure,Holiday/ Rest & Relax,413.812500,1.0,NaN,...,0,1,0,0,0,0,0,0,0,0
11205,16875.0,2018,7,Colombia,NaN,Leisure,Holiday/ Rest & Relax,347.603923,3.0,NaN,...,0,0,0,0,0,0,0,1,0,0
12291,17961.0,2018,7,Others,NaN,Leisure,Holiday/ Rest & Relax,254.925445,1.0,NaN,...,0,1,0,0,0,0,0,0,0,0
12292,17962.0,2018,7,Others,NaN,Leisure,Holiday/ Rest & Relax,536.253543,1.0,NaN,...,0,0,0,0,0,0,0,0,0,0
12293,17963.0,2018,7,Others,NaN,Leisure,Holiday/ Rest & Relax,849.885327,1.0,NaN,...,0,0,0,0,0,0,0,0,0,0
12775,18445.0,2018,7,United States,NaN,Leisure,Holiday/ Rest & Relax,646.217595,3.0,NaN,...,0,1,0,0,0,0,0,0,0,0
17840,2660.0,2018,10,South Korea,NaN,Leisure,Holiday/ Rest & Relax,1072.125000,NaN,NaN,...,0,1,0,0,0,0,0,0,0,0
20528,5348.0,2018,11,Ireland,NaN,Leisure,Sightseeing/ Attractions,641.000000,2.0,NaN,...,0,1,0,0,0,0,0,0,0,0


In [127]:
# Filter rows where all three terminal fields are missing
no_terminal_df = df_trans[
    df_trans['Air_Terminal'].isnull() &
    df_trans['Sea_Terminal'].isnull() &
    df_trans['Land_Terminal'].isnull()
]

# Show the result
print(f"Number of rows with NO terminal data: {len(no_terminal_df)}")
display(no_terminal_df)


Number of rows with NO terminal data: 3


,case,year,month,Country of Residence,City of Residence,Purpose of Visit,Main Purpose of Visit,Weights_QTR,Air_Terminal,Sea_Terminal,...,Travel companion - Alone,Travel companion - Spouse,Travel companion - Your Child/Children,Travel companion - Parents/Parents-in-law,Travel companion - Grandparents/Grandparents-in-law,Travel companion - Siblings,Travel companion - Other relatives,Travel companion - Friends,Travel companion - Business associates/Colleagues,Travel companion - Others
695,699.0,2018,1,Vietnam,Hanoi,Leisure,Holiday/ Rest & Relax,1001.000000,NaN,NaN,...,0,1,0,0,0,0,0,0,0,1
806,811.0,2018,1,Thailand,Chiang Mai,Leisure,Visiting friends/ relatives (who are not inter...,625.809524,NaN,NaN,...,0,0,0,1,0,0,0,0,0,0
819,824.0,2018,1,Italy,Others,Leisure,Holiday/ Rest & Relax,514.500000,NaN,NaN,...,0,1,1,0,0,0,0,0,0,0


In [128]:
#removing city of residence null values - 12 cases
df_cleaned = df_trans.dropna(subset=['City of Residence'])
df_cleaned['City of Residence'].isnull().sum()

0

In [129]:
df_cleaned = df_cleaned[
    ~(df_cleaned['Air_Terminal'].isnull() & df_cleaned['Sea_Terminal'].isnull() & df_cleaned['Land_Terminal'].isnull())
].copy()

In [130]:
# Filter rows where all three terminal fields are missing
no_terminal_df = df_cleaned[
    df_cleaned['Air_Terminal'].isnull() &
    df_cleaned['Sea_Terminal'].isnull() &
    df_cleaned['Land_Terminal'].isnull()
]

# Show the result
print(f"Number of rows with NO terminal data: {len(no_terminal_df)}")
display(no_terminal_df)


Number of rows with NO terminal data: 0


,case,year,month,Country of Residence,City of Residence,Purpose of Visit,Main Purpose of Visit,Weights_QTR,Air_Terminal,Sea_Terminal,...,Travel companion - Alone,Travel companion - Spouse,Travel companion - Your Child/Children,Travel companion - Parents/Parents-in-law,Travel companion - Grandparents/Grandparents-in-law,Travel companion - Siblings,Travel companion - Other relatives,Travel companion - Friends,Travel companion - Business associates/Colleagues,Travel companion - Others


In [131]:
# make the data into datetime

df_cleaned['date'] = pd.to_datetime(dict(year=df_cleaned['year'], month=df_cleaned['month'], day=1))

In [132]:
df_cleaned[['year', 'month', 'date']].describe()

,year,month,date
count,22959.0,22959.000000,22959
mean,2018.0,6.497452,2018-06-16 10:17:55.343002880
min,2018.0,1.000000,2018-01-01 00:00:00
25%,2018.0,4.000000,2018-04-01 00:00:00
50%,2018.0,7.000000,2018-07-01 00:00:00
75%,2018.0,9.000000,2018-09-01 00:00:00
max,2018.0,12.000000,2018-12-01 00:00:00
std,0.0,3.460224,NaN


## changing the travel companion to following
- travel alone = Travel companion - Alone
- travel with family = spouse, parents, grandparents, silbings
- travel with friends
- travel with colleagues
- travel with others

In [134]:
# Create a new column 'Travel_With_Family'
df_cleaned['Travel_With_Family'] = (
    (df_cleaned['Travel companion - Spouse'] == 1) |
    (df_cleaned['Travel companion - Parents/Parents-in-law'] == 1) |
    (df_cleaned['Travel companion - Grandparents/Grandparents-in-law'] == 1) |
    (df_cleaned['Travel companion - Siblings'] == 1)|
    (df_cleaned['Travel companion - Other relatives'] == 1)
).astype(int)

# Check result
print(df_cleaned['Travel_With_Family'].value_counts())

Travel_With_Family
1    11779
0    11180
Name: count, dtype: int64


In [135]:
def classify_transport(row):
    if pd.notnull(row['Air_Terminal']):
        return 'Air'
    elif pd.notnull(row['Sea_Terminal']):
        return 'Sea'
    elif pd.notnull(row['Land_Terminal']):
        return 'Land'
    else:
        return 'Other'

# Apply the function to create a new column
df_cleaned['Main_Transport_Mode'] = df_cleaned.apply(classify_transport, axis=1)


In [136]:
df_cleaned['Main_Transport_Mode'].info()

<class 'pandas.core.series.Series'>
Index: 22959 entries, 0 to 22973
Series name: Main_Transport_Mode
Non-Null Count  Dtype 
--------------  ----- 
22959 non-null  object
dtypes: object(1)
memory usage: 358.7+ KB


In [137]:
print(df_cleaned['Main_Transport_Mode'].value_counts())

Main_Transport_Mode
Air     17966
Land     2612
Sea      2381
Name: count, dtype: int64


In [138]:
# remove the following from analysis 
# 
df_cleaned_analysis = df_cleaned.drop(['case', 'year', 'month', 'Travel companion - Spouse', 'Travel companion - Your Child/Children',
                                       'Travel companion - Parents/Parents-in-law', 'Travel companion - Grandparents/Grandparents-in-law',
                                       'Travel companion - Siblings', 'Travel companion - Other relatives', 'Purpose of Visit',
                                       'Occupation.1','Other Designation','Designation (free text)', 'Air_Terminal', 'Sea_Terminal',
                                      'Land_Terminal'], axis=1)

In [139]:
df_cleaned_analysis.columns

Index(['Country of Residence', 'City of Residence', 'Main Purpose of Visit',
       'Weights_QTR', 'First Visit To Singapore', 'Length of Stay',
       'Travel Type', 'Gender', 'Marital Status', 'Occupation',
       'shopping_fash', 'shopping_jewllery', 'shopping_watches',
       'shopping_wellness', 'shopping_food', 'shopping_gifts',
       'shopping_consumertech', 'shopping_antiques', 'shopping_others',
       'shopping_any', 'totacc', 'totfnb', 'tottran', 'totbiz', 'totedu',
       'totmedi', 'tototh', 'totshopping_rep', 'tot.exp', 'c4a_1', 'MainHotel',
       'Travel companion - Alone', 'Travel companion - Friends',
       'Travel companion - Business associates/Colleagues',
       'Travel companion - Others', 'date', 'Travel_With_Family',
       'Main_Transport_Mode'],
      dtype='object')

In [140]:
df_cleaned_analysis.isnull().sum()

Country of Residence                                    0
City of Residence                                       0
Main Purpose of Visit                                   0
Weights_QTR                                             0
First Visit To Singapore                                0
Length of Stay                                          0
Travel Type                                             0
Gender                                                  0
Marital Status                                          0
Occupation                                              0
shopping_fash                                           0
shopping_jewllery                                       0
shopping_watches                                        0
shopping_wellness                                       0
shopping_food                                           0
shopping_gifts                                          0
shopping_consumertech                                   0
shopping_antiq

## Comments and note to self
1. Total spend on Total Reported Shopping Spend (Most likely uses the shopping variables e.g. shopping_fash etc to calucate this)
2. Travel companion - others most likely gf/bf etc - can be not engaged.
4. clustering model most likely do not need to look at shopping, just focus on total expenses
5. ideas for feature - tot.exp / Length of Stay (understand daily expenses), age band (1-19, 20-40 etc), length of stay (< 3 days, 4 to 7 days, 7 days to 14 days, > 14 days)
6. change date and time

In [142]:
df_cleaned_analysis['Length of Stay'].describe()

count    22959.000000
mean         3.418877
std          3.311082
min          0.000000
25%          2.000000
50%          3.000000
75%          4.000000
max         60.000000
Name: Length of Stay, dtype: float64

In [143]:
df_cleaned_analysis['Length of Stay'].value_counts()

Length of Stay
3     5485
2     4386
4     4302
1     2376
5     2338
0     1741
6      825
7      708
8      164
10     101
14      93
9       87
11      49
12      35
13      32
30      24
15      23
27      17
21      15
29      15
20      15
19      14
17      13
18      12
23      10
16       8
24       8
31       6
28       6
26       5
25       5
57       4
22       4
60       4
56       3
32       3
51       2
43       2
38       2
49       2
46       2
48       2
58       1
40       1
36       1
42       1
50       1
59       1
34       1
53       1
55       1
35       1
37       1
Name: count, dtype: int64

In [144]:
# given that the spread of the data, length of stay will be categorised accordingly
def encode_stay(length):
    if pd.isnull(length):
        return None
    elif length <= 3:
        return 0  # Short Stay
    elif 4 <= length <= 7:
        return 1  # Medium Stay
    else:
        return 2  # Long Stay

df_cleaned_analysis['Stay_Length_Code'] = df_cleaned_analysis['Length of Stay'].apply(encode_stay)


In [145]:
df_cleaned_analysis['Stay_Length_Code'].value_counts()

Stay_Length_Code
0    13988
1     8173
2      798
Name: count, dtype: int64

In [146]:
# Replace 0 with 1 in Length of Stay
df_cleaned_analysis['Length of Stay_fixed'] = df_cleaned_analysis['Length of Stay'].replace(0, 1)

# important to note - anyone who steps into singapore as long as the length of day is 0 I will change it to 1

# Calculate average daily expense using the fixed column
df_cleaned_analysis['avg_daily_expense'] = df_cleaned_analysis['tot.exp'] / df_cleaned_analysis['Length of Stay_fixed']


In [147]:
# Step 1: Calculate average daily expense
df_cleaned_analysis['avg_daily_expense'] = df_cleaned_analysis['tot.exp'] / df_cleaned_analysis['Length of Stay_fixed']



In [148]:
df_cleaned_analysis['avg_daily_expense'].describe()

count    22959.000000
mean       360.307527
std        484.280639
min          0.000000
25%        123.361667
50%        258.182500
75%        444.670000
max      29206.420000
Name: avg_daily_expense, dtype: float64

In [149]:
df_cleaned_analysis.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22959 entries, 0 to 22973
Data columns (total 41 columns):
 #   Column                                             Non-Null Count  Dtype         
---  ------                                             --------------  -----         
 0   Country of Residence                               22959 non-null  object        
 1   City of Residence                                  22959 non-null  object        
 2   Main Purpose of Visit                              22959 non-null  object        
 3   Weights_QTR                                        22959 non-null  float64       
 4   First Visit To Singapore                           22959 non-null  object        
 5   Length of Stay                                     22959 non-null  int64         
 6   Travel Type                                        22959 non-null  object        
 7   Gender                                             22959 non-null  object        
 8   Marital Status       

In [150]:
df_cleaned_analysis['Travel_Type_Code'] = 0
df_cleaned_analysis.loc[df_cleaned_analysis['Travel Type'] == 'Packaged', 'Travel_Type_Code'] = 1


In [151]:
df_cleaned_analysis['Gender_Code'] = 0
df_cleaned_analysis.loc[df_cleaned_analysis['Gender'] == 'Female', 'Gender_Code'] = 1


In [152]:
df_cleaned_analysis['Marital_Status_Code'] = None
df_cleaned_analysis.loc[df_cleaned_analysis['Marital Status'] == 'Single', 'Marital_Status_Code'] = 0
df_cleaned_analysis.loc[df_cleaned_analysis['Marital Status'] == 'Married', 'Marital_Status_Code'] = 1
df_cleaned_analysis.loc[df_cleaned_analysis['Marital Status'] == 'Divorced', 'Marital_Status_Code'] = 2


In [153]:
df_cleaned_analysis['Occupation'].value_counts()

Occupation
Other White collar (teacher, nurse, secretary,                3952
Executive (sales, administration)                             3439
Manager (CEO, company director, senior manager)               3104
Professionals (doctor, lawyer, lecturer, etc)                 2681
Homemaker (full time)                                         2002
Businessman (small company, <50 people)                       1919
Student                                                       1700
Blue collar (technician, hairdresser, hawker, taxi drivers    1360
Retired                                                       1245
Businessman (medium size company, 50 -250 people)              550
Unemployed                                                     501
Others (specify)                                               287
Businessman (large company, > 250 people)                      219
Name: count, dtype: int64

In [154]:
# Create occupation_based column
df_cleaned_analysis['occupation_coded'] = None

# Group 1 → Code 1: Non-working group
df_cleaned_analysis.loc[
    df_cleaned_analysis['Occupation'].isin([
        'Retired',
        'Student',
        'Homemaker (full time)',
        'Unemployed',
        'Others (specify)'
    ]),
    'occupation_coded'
] = 1

# Group 2 → Code 2: Working professionals (white collar, executive, blue collar, etc.)
df_cleaned_analysis.loc[
    df_cleaned_analysis['Occupation'].isin([
        'Other White collar (teacher, nurse, secretary,',
        'Executive (sales, administration)',
        'Manager (CEO, company director, senior manager)',
        'Professionals (doctor, lawyer, lecturer, etc)',
        'Blue collar (technician, hairdresser, hawker, taxi drivers'
    ]),
    'occupation_coded'
] = 2

# Group 3 → Code 3: Business owners
df_cleaned_analysis.loc[
    df_cleaned_analysis['Occupation'].isin([
        'Businessman (small company, <50 people)',
        'Businessman (medium size company, 50 -250 people)',
        'Businessman (large company, > 250 people)'
    ]),
    'occupation_coded'
] = 3


In [155]:
df_cleaned_analysis.columns

Index(['Country of Residence', 'City of Residence', 'Main Purpose of Visit',
       'Weights_QTR', 'First Visit To Singapore', 'Length of Stay',
       'Travel Type', 'Gender', 'Marital Status', 'Occupation',
       'shopping_fash', 'shopping_jewllery', 'shopping_watches',
       'shopping_wellness', 'shopping_food', 'shopping_gifts',
       'shopping_consumertech', 'shopping_antiques', 'shopping_others',
       'shopping_any', 'totacc', 'totfnb', 'tottran', 'totbiz', 'totedu',
       'totmedi', 'tototh', 'totshopping_rep', 'tot.exp', 'c4a_1', 'MainHotel',
       'Travel companion - Alone', 'Travel companion - Friends',
       'Travel companion - Business associates/Colleagues',
       'Travel companion - Others', 'date', 'Travel_With_Family',
       'Main_Transport_Mode', 'Stay_Length_Code', 'Length of Stay_fixed',
       'avg_daily_expense', 'Travel_Type_Code', 'Gender_Code',
       'Marital_Status_Code', 'occupation_coded'],
      dtype='object')

In [161]:
df_cleaned_analysis.info()

,Country of Residence,City of Residence,Main Purpose of Visit,Weights_QTR,First Visit To Singapore,Length of Stay,Travel Type,Gender,Marital Status,Occupation,...,date,Travel_With_Family,Main_Transport_Mode,Stay_Length_Code,Length of Stay_fixed,avg_daily_expense,Travel_Type_Code,Gender_Code,Marital_Status_Code,occupation_coded
0,India,Delhi,Holiday/ Rest & Relax,660.046512,Yes,6,Packaged,Male,Married,"Executive (sales, administration)",...,2018-01-01,1,Air,1,6,246.816667,1,0,1,2
1,India,Delhi,Visiting friends/ relatives (who are not inter...,433.217949,No,50,Non-packaged,Female,Single,Student,...,2018-01-01,0,Air,2,50,1.982000,0,1,0,1
2,United Arab Emirates,Dubai,General business purpose,335.125000,No,6,Business (Non-packaged),Male,Single,"Manager (CEO, company director, senior manager)",...,2018-01-01,0,Air,1,6,881.045000,0,0,0,2
3,India,Others,Holiday/ Rest & Relax,490.977273,No,7,Non-packaged,Male,Married,"Professionals (doctor, lawyer, lecturer, etc)",...,2018-01-01,1,Air,1,7,96.357143,0,0,1,2
4,Canada,Toronto,Visiting an international student (relative or...,379.800000,Yes,2,Non-packaged,Male,Married,"Professionals (doctor, lawyer, lecturer, etc)",...,2018-01-01,0,Air,0,2,2091.275000,0,0,1,2


In [156]:
# Feature creation

SyntaxError: invalid syntax (2373907948.py, line 1)

In [ ]:
# understand their spending behaviours and preferences 
